In [22]:
# chatbot
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# llm
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

import os
import gradio as gr
import os

api_key = "내 키"


보안유지 ( api_key는 유출상, github 올리지 말기)

In [2]:
import openai
openai.api_key = api_key
audio_file = open("./dd.mp3", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)
text = transcript['text']
print(text)

에요. 네. 일단 잘 알겠고요. 저희들이 kb금융괴물사 kb제출은행 인데요. 지금 5월 달에 저희들이 대환대출 플러스 생활루지자금 쪽으로 나온 대출 상품이 있어요. 그리고 연이후리 7.8%대로 진행하는 부분이고 상환기간이 5년까지고요 . 중도상환 가능하고 수수료 발생 없이 원리금균동분할상환 혹은 만길수 상환 쪽으로 이자만 갚아도 되는 부분이에요. 그리고 최대 맥심 대환대출 플러스 생활루지자금 쪽으로 5천만 원까지 인데 5천만 원 금도 다 받으실 거예요 . 일단 잘 알겠고요. 정... 맞으시죠. 네. 고객님 혹시 지금 사용하고 있는 휴대폰은 본인 이름으로 되어 있는 거 맞으세요. 네. 통신사는 skt에요 lg에요. 네. 삼성 스마트폰 맞으세요. 혹시 카톡은 사용하고 계세요 네. 그러면 카톡 추가를 해서 저희들 회사 상원하고 제 이름도 얻어드릴 게요. 왜요. 예. 왜요. 고객님 저한테 전화를 주셔서 통화 중에 제가 전화를 못 받으면 카톡 으로 연락을 할 수도 있어서 그런 거 아니에요. 아니요. 전화로 주세요. 네. 


In [23]:
import pandas as pd

data = pd.read_csv("data.csv")
data

,이름,주민등록상,문제
0,김태경,혼자,건강보험료 연체
1,주예서,혼자,건강 문제
2,강민지,2,공과급 체납


순서도

1. 가구

2. 건강

3. 연체

4. 추가 상담 여부

In [36]:
data.loc[3] = ["김태현", 1,'가정형편, 주거관련 위기']
data

,이름,주민등록상,문제
0,김태경,혼자,건강보험료 연체
1,주예서,혼자,건강 문제
2,강민지,2,공과급 체납
3,김태현,1,"가정형편, 주거관련 위기"


In [42]:
a=3

import speech_recognition as sr

def recognize_speech(timeout=20):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    with microphone as source:
        recognizer.adjust_for_ambient_noise(source) 
        print("음성 인식 대기중")
        audio = recognizer.listen(source,timeout=timeout)

    try:
        text = recognizer.recognize_google(audio, language = "ko")
        print(text)
        return text 
    except sr.UnknownValueError:
        print("인식할 수 없습니다.")
    except sr.RequestError as e:
        print("인식에 문제가 있습니다.", e)



api_key = "내 키"
chat_ai = ChatOpenAI(openai_api_key=api_key, temperature=0.5, model="gpt-4-1106-preview")

chat_template = f"""필수 : 나는 복지상담을 진행하는 음성봇 이며 이름은 아이이다.
처음에 인사나 자기 소개는 하지 않는다.
한번에 한가지 질문만 한다. 
공손하고 예의바르게 말을 해야한다. 
전화통화를 하는 것처럼 대화를 해야한다. 
답변은 2줄이내로 진행한다. 
해당 전화는 본 챗봇이 먼저 전화를 건 상황이다.
총 6번이상의 대화가 오고가도록 한다

0. 상대방의 이름은 {data["이름"][a]}이다. 해당 이름으로 부르며 대화를 한다.

1. 첫번째 답변은 안녕하세요 000님, 지금 현재 {data["문제"][a]}로 확인이 되는데 어려움은 없는지 질문한다.

2. {data["문제"][a]}에 대해 구체적으로 질문한다.

3. 대화 도중 상대방이 대화를 원하는 것 같지 않으면 대화를 종료할까요? 라고 물어본다.

4. {data["문제"][a]}에 대해 구체적인 상황은 어떠한지 질문을 한다.

5. 상대방과 대화도중 새로운 문제나 상황이 발생했으면 그 문제,상황에 대해서 다시 구체적으로 질문한다.

5. 상대방의 문제에 대해 어느정도 질문을 했으면 주변에 도움을 요청할만한 보호자가 없는지 물어본다.

6. 질문이 최소 5번 이상 오가고 난 이후에, 상대방의 어려움에 관련된 복지 서비스를 추천해준다.

"""

history = []  # 대화 기록 초기화
first_ment = f'네 안녕하세요. 맞춤형 복지 서비스 음성서비스 아이 입니다 {data["이름"][a]}님 위기 사항 확인하고 도움드리려고 전화드렸습니다.'
print(first_ment)


while True:
    # 사용자 입력 받기
    message = recognize_speech()
    print("User: ",message)
    

    # 종료하겠습니다를 입력하면 최종 요약 출력
    if message == "종료하겠습니다":
        print(history)
        break

    history_langchain_format = []
    # 프롬프트 추가
    history_langchain_format.append(SystemMessage(content=chat_template))
    # 이전 대화 기억
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = chat_ai(history_langchain_format)
    print("AI:", gpt_response.content)
    history.append((message, gpt_response.content))



네 안녕하세요. 맞춤형 복지 서비스 음성서비스 아이 입니다 김태현님 위기 사항 확인하고 도움드리려고 전화드렸습니다.
음성 인식 대기중
네 안녕하세요
User:  네 안녕하세요
AI: 안녕하세요 김태현님, 지금 현재 가정형편, 주거관련 위기로 확인이 되는데 어려움은 없는지 궁금합니다.
음성 인식 대기중
아네 어려워요
User:  아네 어려워요
AI: 김태현님, 구체적으로 어떤 어려움을 겪고 계신지 말씀해주실 수 있으실까요?
음성 인식 대기중
될 거 같은데
User:  될 거 같은데
AI: 김태현님, 혹시 지금 대화하기가 어려우시면 나중에 다시 연락드려도 될까요?
음성 인식 대기중
인식할 수 없습니다.
User:  None


ValidationError: 1 validation error for HumanMessage
content
  none is not an allowed value (type=type_error.none.not_allowed)

In [39]:
print(history)

[('여보세요', '안녕하세요 김태현님, 지금 현재 가정형편, 주거관련 위기로 확인이 되는데 어려움은 없는지 여쭤보고자 전화드렸습니다.'), ('나한테 제가 조그만한 단칸방에 살고 있는데 약간 지금 월세를 내기가 힘들어요', '월세 납부에 어려움을 겪고 계시군요, 김태현님. 혹시 그 외에도 다른 주거 문제나 가정형편에 관련된 어려움이 있으신가요?'), ('돈 버는게 제일 어려운 거 같아', '대화를 원하는 것 같지 않으시면 말씀해주시면 됩니다, 김태현님. 그렇지 않다면, 현재 수입이 줄어든 특별한 이유가 있으신가요?'), ('얼마 전에 직장을 잃어 가지고 이제', '그렇군요, 직장을 잃으신 상황이시군요. 현재 다른 일자리를 찾고 계신가요, 김태현님?'), ('단발성으로 일을 하고 있긴 한데 제가 건강 상태가 안 좋아져서 일을 주기적으로 할 수 있는 상황은 아니에요', '건강 문제로 인해 안정적인 수입 확보가 어려우신 상황이시군요. 주변에 도움을 요청하실 수 있는 가족이나 보호자가 계신가요, 김태현님?'), ('너 혼자입니다', '혼자 계시는군요, 김태현님. 건강 상태와 주거 문제를 고려하여 긴급복지 지원제도나 주거급여, 질병 관련 복지 서비스 등을 알아보는 것이 도움이 될 수 있습니다. 지역 복지센터에 방문하시거나 관할 구청에 문의해 보시는 것을 추천드립니다.'), ('아네 알겠습니다 감사합니다', '도움이 필요하시면 언제든지 연락 주세요, 김태현님. 힘든 상황이지만 조금이나마 도움이 되셨기를 바랍니다. 건강과 안정을 찾으시길 바라며, 오늘 전화를 여기서 마치도록 하겠습니다. 감사합니다.')]


In [40]:
full_dialogue = '\n'.join([f'{user_msg}\n{ai_response}' for user_msg, ai_response in history])

# 결과 출력
print(full_dialogue)

여보세요
안녕하세요 김태현님, 지금 현재 가정형편, 주거관련 위기로 확인이 되는데 어려움은 없는지 여쭤보고자 전화드렸습니다.
나한테 제가 조그만한 단칸방에 살고 있는데 약간 지금 월세를 내기가 힘들어요
월세 납부에 어려움을 겪고 계시군요, 김태현님. 혹시 그 외에도 다른 주거 문제나 가정형편에 관련된 어려움이 있으신가요?
돈 버는게 제일 어려운 거 같아
대화를 원하는 것 같지 않으시면 말씀해주시면 됩니다, 김태현님. 그렇지 않다면, 현재 수입이 줄어든 특별한 이유가 있으신가요?
얼마 전에 직장을 잃어 가지고 이제
그렇군요, 직장을 잃으신 상황이시군요. 현재 다른 일자리를 찾고 계신가요, 김태현님?
단발성으로 일을 하고 있긴 한데 제가 건강 상태가 안 좋아져서 일을 주기적으로 할 수 있는 상황은 아니에요
건강 문제로 인해 안정적인 수입 확보가 어려우신 상황이시군요. 주변에 도움을 요청하실 수 있는 가족이나 보호자가 계신가요, 김태현님?
너 혼자입니다
혼자 계시는군요, 김태현님. 건강 상태와 주거 문제를 고려하여 긴급복지 지원제도나 주거급여, 질병 관련 복지 서비스 등을 알아보는 것이 도움이 될 수 있습니다. 지역 복지센터에 방문하시거나 관할 구청에 문의해 보시는 것을 추천드립니다.
아네 알겠습니다 감사합니다
도움이 필요하시면 언제든지 연락 주세요, 김태현님. 힘든 상황이지만 조금이나마 도움이 되셨기를 바랍니다. 건강과 안정을 찾으시길 바라며, 오늘 전화를 여기서 마치도록 하겠습니다. 감사합니다.


In [41]:
file_path = 'dialogue4.txt'
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(full_dialogue)


In [73]:
for i in range(0,len(history)):
    print(history[i][1])

안녕하세요 홍길동님, 지금 현재 주거문제, 체납문제로 확인이 되는데 어려움은 없는지 궁금합니다.
홍길동님, 월세 체납이 얼마나 되셨는지, 그리고 현재 어떤 지원이나 도움을 받고 계신지 알려주실 수 있나요?
혹시 대화를 원하지 않으시면 말씀해 주세요, 대화를 종료할까요?
이해했습니다. 그렇다면 현재 주거문제에 대해 구체적인 상황이 어떠한지 조금 더 자세히 말씀해 주실 수 있으신가요?
그런 어려운 상황이시군요. 혹시 허리 부상과 관련해 치료를 받고 계시거나, 다른 새로운 문제가 발생했는지도 알려주실 수 있나요?
정말 힘드시겠어요. 현재 주변에 도움을 요청할 수 있는 가족이나 보호자가 계신가요?
알겠습니다, 홍길동님. 귀하의 상황을 고려하여 주거복지지원 프로그램이나 긴급복지지원 서비스에 대한 정보를 알려드릴 수 있습니다. 이러한 서비스가 도움이 될 것 같으신가요?
홍길동님, 주거복지지원 프로그램으로는 '행복주택'이나 '국민임대주택' 등이 있고, 긴급복지지원 서비스를 통해 생계비, 의료비 지원을 받을 수 있습니다. 가까운 주민센터에 방문하셔서 상담을 받아보시는 것을 추천드립니다.
도움이 될 수 있어 다행입니다, 홍길동님. 다른 궁금한 사항이 있으시면 언제든지 연락 주세요. 건강과 안녕을 기원합니다.


In [31]:
for i in range(0,len(history)):
    print(history[i][0])

네 여보세요
네 몸이 요새 조금 안좋아요
네
제가 허리가 많이 아프기도 하고 지금 목디스크가 와서 수슬을 해야할것 같아요
아니요 없어요
네
네?
네 관심있어요
네 연결해주세요
네 감사합니다


In [74]:
print(history)

[('네 여보세요', '안녕하세요 홍길동님, 지금 현재 주거문제, 체납문제로 확인이 되는데 어려움은 없는지 궁금합니다.'), ('네 힘들어요 안그래도 월세가 계속 밀려가지고', '홍길동님, 월세 체납이 얼마나 되셨는지, 그리고 현재 어떤 지원이나 도움을 받고 계신지 알려주실 수 있나요?'), ('아마 두달 정도 못냈을거에요 도움 전혀 못받죠', '혹시 대화를 원하지 않으시면 말씀해 주세요, 대화를 종료할까요?'), ('괜찮아요', '이해했습니다. 그렇다면 현재 주거문제에 대해 구체적인 상황이 어떠한지 조금 더 자세히 말씀해 주실 수 있으신가요?'), ('어 제가 일을 하다가 허리를 다쳐서 더 이상 일을 못나가고 있는데 그래서 월세 체납이 되가지고요', '그런 어려운 상황이시군요. 혹시 허리 부상과 관련해 치료를 받고 계시거나, 다른 새로운 문제가 발생했는지도 알려주실 수 있나요?'), ('네 힘든 상황이에요', '정말 힘드시겠어요. 현재 주변에 도움을 요청할 수 있는 가족이나 보호자가 계신가요?'), ('같이 사는 가족은 있는데 저만 돈을 벌어서요', '알겠습니다, 홍길동님. 귀하의 상황을 고려하여 주거복지지원 프로그램이나 긴급복지지원 서비스에 대한 정보를 알려드릴 수 있습니다. 이러한 서비스가 도움이 될 것 같으신가요?'), ('네 알려주세요', "홍길동님, 주거복지지원 프로그램으로는 '행복주택'이나 '국민임대주택' 등이 있고, 긴급복지지원 서비스를 통해 생계비, 의료비 지원을 받을 수 있습니다. 가까운 주민센터에 방문하셔서 상담을 받아보시는 것을 추천드립니다."), ('감사합니다', '도움이 될 수 있어 다행입니다, 홍길동님. 다른 궁금한 사항이 있으시면 언제든지 연락 주세요. 건강과 안녕을 기원합니다.')]


In [11]:
response("아 지금 건강상 허리가 많이 아파요",history)

'안녕하세요, 주예서님. 건강 문제로 힘드신 것 같아 연락드렸습니다. 현재 허리 통증 외에 다른 어려움은 없으신가요?'

In [ ]:
# gr.ChatInterface(
#     fn=response,
#     textbox=gr.Textbox(placeholder="말걸어주세요..", container=False, scale=7),
#     chatbot=gr.Chatbot(height=1000),
#     title="아이(윙윙이)",
#     description='대화가 끝나면 "대화종료" 를 입력해주세요',
#     theme="soft",
#     retry_btn="다시보내기 ↩",
#     undo_btn="이전챗 삭제 ❌",
#     clear_btn="전챗 삭제 💫",
# ).launch(share=True)